In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

import time

from openpyxl import Workbook, load_workbook
from tqdm import tqdm

def get_driver():
    chrome_options = webdriver.ChromeOptions()
    version = '115.0.5790.111'
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager(version).install()), 
                              options=chrome_options)
    except:
        driver = webdriver.Chrome()
    return driver

def send_key(_xpath, _txt):
    driver.find_element(By.XPATH, _xpath).clear()
    time.sleep(0.1)
    driver.find_element(By.XPATH, _xpath).send_keys(_txt)

    
def click(_xpath):
    driver.find_element(By.XPATH, _xpath).click()
    
    
def get_text(_xpath):
    return driver.find_element(By.XPATH, _xpath).text.strip()

def save_excel(_FILENAME, _DATA, _HEADER):
    book = Workbook()
    sheet = book.active
    sheet.title = 'result'
    sheet.append(_HEADER)
    sheet.column_dimensions['A'].width = 30
    sheet.column_dimensions['B'].width = 50
    sheet.column_dimensions['C'].width = 200
    for data in _DATA:
        sheet.append(data)
    book.save(_FILENAME)
    
def load_excel(_FILENAME):
    tEx = load_workbook(_FILENAME)
    # 시트를 불러오는 두가지 방법
    #sheet1 = tEx['Sheet1']
    sheet1 = tEx.active
    data_list = []
    for row in sheet1.rows:
        #cell_example = row[0].value
        tmp = [cell.value for cell in row]
        data_list.append(tmp)
    return data_list

In [28]:
driver = get_driver()
driver.get('https://www.quora.com/search?q=virtual%20influencer')
WANT_MIN_NUM = 20 # 20개 아티클
SAVE_FILE = '테스트.xlsx'

# Quora 링크 + 대표답글 1개 가져오기 

In [87]:
while True:
    bs = BeautifulSoup(driver.page_source, 'lxml')
    divs = bs.find_all('div', class_='q-box qu-borderBottom qu-p--medium qu-pb--tiny')
    if WANT_MIN_NUM > len(divs):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        body.send_keys(Keys.PAGE_DOWN)
    else:
        break

In [105]:
action = ActionChains(driver)
more_xpath = '//*[@id="mainContent"]/div/div/div[2]/div[{}]/div/div[1]/div/div[1]/div[2]/div/div/div[2]/div'

for i in tqdm(range(1, len(divs)+1)):
    try:
        element = driver.find_element(By.XPATH, more_xpath.format(i))
        action.move_to_element(element).click().perform()
    except NoSuchElementException:
        pass
    time.sleep(0.2)

100%|██████████| 23/23 [00:04<00:00,  4.74it/s]


In [103]:
"""
Title Link 답변 
"""
data_list = [] 
bs = BeautifulSoup(driver.page_source, 'lxml')
divs = bs.find_all('div', class_='q-box qu-borderBottom qu-p--medium qu-pb--tiny')
for div in divs:
    a_tag = div.find('a', class_='q-box Link___StyledBox-t2xg9c-0 dFkjrQ puppeteer_test_link qu-display--block qu-cursor--pointer qu-hover--textDecoration--underline')
    타이틀 = a_tag.text.strip()
    링크 = a_tag['href']
    p_tags = div.find_all('p', class_='q-text qu-display--block qu-wordBreak--break-word qu-textAlign--start')
    본문 = ''
    for p in p_tags:
        본문 += p.text.strip()
    data_list.append([타이틀, 링크, 본문])

In [104]:
save_excel(SAVE_FILE, data_list, ['타이틀', "링크", "본문"])

# 링크들어가서 전체 답변 가져오기

In [30]:
excel = load_excel(SAVE_FILE)

In [35]:
data_list = []
for row in tqdm(excel[1:]):
    링크 = row[1]
    driver.get(링크)
    time.sleep(2)
    # All related -> Answer 부분으로 바꾸는거 
    #'//*[@id="mainContent"]/div[2]/div/div[1]/div/div/button/div'
    try: 
        click('//*[@id="mainContent"]/div[3]/div/div[1]/div/div/button/div/div')
        time.sleep(1)
        click('//*[@id="mainContent"]/div[3]/div/div[1]/div/div[2]/div/div[1]/div/div/div[2]/div')
    except: # 여기가 안되면 답변이 X
        continue
    time.sleep(1)

    # 댓글부분 읽어오기
    bs = BeautifulSoup(driver.page_source, 'lxml')
    anser_xpath = "q-box dom_annotate_question_answer_item_{} qu-borderAll qu-borderRadius--small qu-borderColor--raised qu-boxShadow--small qu-mb--small qu-bg--raised"

    idx = 0 
    while True:
        div = bs.find('div', class_=anser_xpath.format(idx))
        try:
            본문 = div.find('div', class_='q-box spacing_log_answer_content puppeteer_test_answer_content')
        except:
            break
        data_list.append([링크, 본문.text.strip()])
        idx += 1

100%|██████████| 23/23 [01:54<00:00,  4.98s/it]


In [36]:
save_excel('전체댓글_'+SAVE_FILE, data_list, ['링크',  "댓글"])